
# IMU-basierte Herz- und Atemratenanalyse aus der Sohle (unter dem Fußgewölbe)

Dieses Notebook analysiert IMU-Daten aus der Sohle, um:

- die Abtastrate zu bestimmen,
- das Beschleunigungssignal aufzubereiten,
- eine Herzfrequenzschätzung aus einem definierten Frequenzband zu berechnen,
- eine grobe Atemfrequenzschätzung zu liefern,
- Spektren und Zeitverläufe zu visualisieren.

> **Hinweis:** Passe `DATA_PATH` und ggf. Spaltennamen (`COL_TIME`, `COL_AX`, `COL_AY`, `COL_AZ`) an deine CSV-Datei an.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import signal


In [ ]:

# === Pfad & Spaltennamen anpassen ===

DATA_PATH = "sitzen01.csv"  # Pfad zur CSV-Datei (lokal anpassen)

COL_TIME = "t_ms"           # Zeitstempel in Millisekunden (oder NaN, falls nicht vorhanden)
COL_AX   = "ax"             # Beschleunigung X
COL_AY   = "ay"             # Beschleunigung Y
COL_AZ   = "az"             # Beschleunigung Z

# Frequenzbänder (in Hz)
HR_BAND   = (0.8, 2.5)      # 48–150 bpm
RESP_BAND = (0.1, 0.6)      # 6–36 Atemzüge/min


In [ ]:

# === Daten laden ===

df = pd.read_csv(DATA_PATH)

print("Spalten:", df.columns.tolist())
print("Erste Zeilen:")
display(df.head())

# Falls 'seq' existiert, als Index nutzen (optional)
if "seq" in df.columns:
    df = df.set_index("seq")

# Zeitvektor berechnen
if COL_TIME in df.columns:
    t = df[COL_TIME].to_numpy().astype(float) / 1000.0  # ms -> s
else:
    # Fallback: gleichmäßig verteilte Zeit, falls keine Zeitspalte
    t = np.arange(len(df), dtype=float)


In [ ]:

# === Abtastrate schätzen ===

# Wenn echte Zeitstempel vorhanden
dt = np.diff(t)
dt_valid = dt[np.isfinite(dt) & (dt > 0)]

if len(dt_valid) > 0:
    dt_median = np.median(dt_valid)
    fs_est = 1.0 / dt_median
else:
    dt_median = 1.0
    fs_est = 1.0

print(f"Geschätzte Abtastrate: {fs_est:.2f} Hz (Median Δt = {dt_median*1000:.2f} ms)")

fs = fs_est


In [ ]:

# === Beschleunigung vorbereiten ===

ax = df[COL_AX].to_numpy().astype(float)
ay = df[COL_AY].to_numpy().astype(float)
az = df[COL_AZ].to_numpy().astype(float)

# Betrag der Beschleunigung
amag = np.sqrt(ax**2 + ay**2 + az**2)

# Hochpass, um DC-Anteil (Schwerkraft, Offset) zu entfernen
hp_cut = 0.5  # Hz
b_hp, a_hp = signal.butter(4, hp_cut / (fs / 2.0), btype="highpass")
amag_hp = signal.filtfilt(b_hp, a_hp, amag)

# Normalisieren (optional), um vergleichbare Skalen zu haben
amag_norm = (amag_hp - np.mean(amag_hp)) / (np.std(amag_hp) + 1e-12)

print("Signallänge:", len(amag_norm))


In [ ]:

# === Zeitverlauf visualisieren ===

plt.figure(figsize=(10, 4))
plt.plot(t, amag_norm)
plt.xlabel("Zeit [s]")
plt.ylabel("norm. |a|")
plt.title("Normierter Betrag der Beschleunigung über die Zeit")
plt.tight_layout()
plt.show()

# Zoom auf ein kürzeres Fenster (z.B. 10 Sekunden), um Herzschläge anzusehen
window_duration = 10.0  # Sekunden
if t[-1] > window_duration:
    mask = (t >= t[0]) & (t <= t[0] + window_duration)
    plt.figure(figsize=(10, 4))
    plt.plot(t[mask], amag_norm[mask])
    plt.xlabel("Zeit [s]")
    plt.ylabel("norm. |a|")
    plt.title(f"Zoom: erste {window_duration:.1f} s")
    plt.tight_layout()
    plt.show()


In [ ]:

# === Welch-PSD und Herzfrequenzschätzung ===

# Welch-Parameter
nperseg = int(fs * 8)  # 8 s-Fenster
noverlap = nperseg // 2

f, Pxx = signal.welch(amag_norm, fs=fs, nperseg=nperseg, noverlap=noverlap)

plt.figure(figsize=(10, 4))
plt.semilogy(f, Pxx)
plt.xlim(0, 10)
plt.xlabel("Frequenz [Hz]")
plt.ylabel("PSD")
plt.title("Leistungsspektrum des normierten Betrags")
plt.tight_layout()
plt.show()

# Herzband extrahieren
hr_mask = (f >= HR_BAND[0]) & (f <= HR_BAND[1])
f_hr = f[hr_mask]
P_hr = Pxx[hr_mask]

if len(P_hr) > 0:
    # Dominanter Peak im Herzband
    idx_max = np.argmax(P_hr)
    f_peak_hr = f_hr[idx_max]
    hr_bpm = f_peak_hr * 60.0

    # Einfaches Qualitätsmaß: Peakleistung / Median im Band
    median_band = np.median(P_hr) + 1e-15
    peak_power = P_hr[idx_max]
    quality_ratio = peak_power / median_band

    print(f"Geschätzte Herzfrequenz: {hr_bpm:.1f} bpm (Peak bei {f_peak_hr:.3f} Hz)")
    print(f"Qualitätsmaß (Peak/Median im Herzband): {quality_ratio:.2f}")

    plt.figure(figsize=(10, 4))
    plt.semilogy(f_hr, P_hr)
    plt.axvline(f_peak_hr, linestyle="--")
    plt.xlabel("Frequenz [Hz]")
    plt.ylabel("PSD")
    plt.title("Herzband-PSD")
    plt.tight_layout()
    plt.show()
else:
    print("Herzband enthält keine Frequenzpunkte – prüfe Abtastrate und HR_BAND.")


In [ ]:

# === Atemfrequenzschätzung ===

resp_mask = (f >= RESP_BAND[0]) & (f <= RESP_BAND[1])
f_resp = f[resp_mask]
P_resp = Pxx[resp_mask]

if len(P_resp) > 0:
    idx_max_resp = np.argmax(P_resp)
    f_peak_resp = f_resp[idx_max_resp]
    resp_bpm = f_peak_resp * 60.0

    median_resp = np.median(P_resp) + 1e-15
    peak_resp = P_resp[idx_max_resp]
    quality_ratio_resp = peak_resp / median_resp

    print(f"Geschätzte Atemfrequenz: {resp_bpm:.1f} Atemzüge/min (Peak bei {f_peak_resp:.3f} Hz)")
    print(f"Qualitätsmaß (Peak/Median im Resp.-Band): {quality_ratio_resp:.2f}")

    plt.figure(figsize=(10, 4))
    plt.semilogy(f_resp, P_resp)
    plt.axvline(f_peak_resp, linestyle="--")
    plt.xlabel("Frequenz [Hz]")
    plt.ylabel("PSD")
    plt.title("Respirationsband-PSD")
    plt.tight_layout()
    plt.show()
else:
    print("Respirationsband enthält keine Frequenzpunkte – prüfe Abtastrate und RESP_BAND.")


In [ ]:

# === Grobe SNR-Betrachtung für das Herzband ===

# Gesamtleistung im Herzband
power_hr_band = np.trapz(P_hr, f_hr) if len(P_hr) > 0 else np.nan

# Referenzband (z.B. 3–5 Hz) als „Rauschen“ (hier nur exemplarisch)
noise_band = (3.0, 5.0)
noise_mask = (f >= noise_band[0]) & (f <= noise_band[1])
f_noise = f[noise_mask]
P_noise = Pxx[noise_mask]
power_noise_band = np.trapz(P_noise, f_noise) if len(P_noise) > 0 else np.nan

print(f"Bandpower Herzband ({HR_BAND[0]}–{HR_BAND[1]} Hz): {power_hr_band:.3e}")
print(f"Bandpower Referenzband ({noise_band[0]}–{noise_band[1]} Hz): {power_noise_band:.3e}")

if np.isfinite(power_hr_band) and np.isfinite(power_noise_band) and power_noise_band > 0:
    snr_lin = power_hr_band / power_noise_band
    snr_db = 10 * np.log10(snr_lin)
    print(f"Grobe SNR (Herzband vs Referenzband): {snr_db:.2f} dB")
else:
    print("SNR konnte nicht berechnet werden – prüfe Spektrum und Frequenzbänder.")
